* Data pipeline for the image ETL

In [170]:
from __future__ import absolute_import, division, print_function
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
tf.VERSION
AUTOTUNE = tf.contrib.data.AUTOTUNE

In [171]:
import pathlib
data_root = pathlib.Path('./data/train/')
print(data_root)
for item in data_root.iterdir():
  print(item)

data/train
data/train/snapper
data/train/bream


In [172]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count
all_image_paths[:10]

['data/train/snapper/red-snapper.jpg',
 'data/train/snapper/mutton_snapper.jpg',
 'data/train/bream/yellowfin-bream.jpg']

In [173]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['bream', 'snapper']

In [174]:
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

{'bream': 0, 'snapper': 1}

In [175]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])

First 10 labels indices:  [1, 1, 0]


In [176]:
# import IPython.display as display
# for n in range(2):
#   image_path = random.choice(all_image_paths)
#   display.display(display.Image(image_path))
# #   print(caption_image(image_path))
#   print()

The preprocessing include a resize action. Currently we are using squash method - ignore the ration and turn it to square. Please check [this article](https://forums.fast.ai/t/impact-of-image-resizing-on-model-training-time-and-performance/1980) to see the comparison amoung the different methods of resizing

In [110]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize_images(image, [192, 192])
  image /= 255.0  # normalize to [0,1] range

  return image

In [111]:
def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

Take a look what happen after the image processing.

In [112]:
# import matplotlib.pyplot as plt

# image_path = all_image_paths[1]
# label = all_image_labels[1]

# load_and_preprocess_image(image_path)

# plt.imshow(load_and_preprocess_image(image_path))
# plt.grid(False)
# plt.title(label_names[label].title())
# plt.show()

<tf.Tensor: id=820, shape=(192, 192, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>

Start preparing the dataset

In [113]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

In [157]:
# image_ds = all_image_paths.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
# image_ds
# label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
# label_ds

AttributeError: 'list' object has no attribute 'map'

In [154]:
# for label in label_ds.take(10):
#   print(label_names[label.numpy()])

In [117]:
# image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

In [165]:
train_images = []
train_labels = []
for image_path in all_image_paths:
    train_images.append(load_and_preprocess_image(image_path))
for label_item in tf.cast(all_image_labels, tf.int64):
    train_labels.append(label_item)
ds=tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(2).repeat()
    

Now Make the model ready

In [164]:
# BATCH_SIZE = 32

# # Setting a shuffle buffer size as large as the dataset ensures that the data is
# # completely shuffled.
# ds = image_label_ds.shuffle(buffer_size=image_count)
# ds = ds.repeat()
# ds = ds.batch(BATCH_SIZE)
# # `prefetch` lets the dataset fetch batches, in the background while the model is training.
# ds = ds.prefetch(buffer_size=AUTOTUNE)
# ds

<PrefetchDataset shapes: ((?, 192, 192, 3), (?,)), types: (tf.float32, tf.int64)>

In [166]:
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(192, 192, 3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])


In [167]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [168]:
model.fit(ds, epochs=1, steps_per_epoch=1)

Epoch 1/1
1/1 [==============================] - 0s 101ms/step - loss: 2.4081 - acc: 0.5000


In [169]:
predictions = model.predict(ds, steps=1)
predictions[0]

array([1.0000000e+00, 3.1232432e-19, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)